그라디오 챗봇와 랭체인 LLM 연동하기

In [ ]:
# 그라디오 라이브러리를 설치합니다.
#poetry add gradio 
# 랭체인 라이브러리를 설치합니다.
#poetry add langchain 
# 랭체인 OpenAI 연동 라이브러리를 설치합니다.
#poetry add langchain-openai

#### LangChain의 ChatOpenAI가 환경 변수를 자동으로 로드하는 방식
* load_dotenv()를 실행하면 .env 파일에 있는 환경 변수가 시스템 환경 변수로 등록됨
* LangChain의 ChatOpenAI는 api_key=None일 때 os.getenv("OPENAI_API_KEY")를 자동으로 불러옴
* 따라서 api_key를 명시적으로 설정하지 않아도 API 키가 자동으로 적용됨

In [1]:
from dotenv import load_dotenv
import os

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:4])

gsk_


In [2]:
import langchain

print(langchain.__version__)

0.3.27


In [3]:
from langchain_openai import ChatOpenAI

# LLM 인스턴스 생성
#llm = ChatOpenAI(model_name="gpt-3.5-turbo")
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

response = llm.invoke([{"role": "user", "content": "파이썬이란?"}])
print(response) #AIMessage
print(response.content) #str

c:\Users\KOSTA\AppData\Local\pypoetry\Cache\virtualenvs\mylangchain-app-tQs1zK9u-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


content='파이썬(Python)은 **고급 프로그래밍 언어**로, 1991년 네덜란드의 개발자 **귀도 반 로섬(Guido van Rossum)**이 만들었어요. 이름은 그가 좋아했던 코미디단 **Monty Python**에서 따왔다고 해요.\n\n### 파이썬의 핵심 특징:\n| 특징 | 설명 |\n|------|------|\n| **쉽고 간결하다** | 문법이 단순해서 초보자도 금방 읽고 쓸 수 있어요. |\n| **인터프리터 언어** | 한 줄씩 바로 실행 가능해서 테스트와 디버깅이 쉬워요. |\n| **무료/오픈소스** | 누구나 무료로 설치하고, 소스 코드도 확인·수정할 수 있어요. |\n| **크로스플랫폼** | Windows, macOS, Linux 등 어디서든 돌아가요. |\n| **방대한 생태계** | AI, 데이터분석, 웹, 자동화 등 **라이브러리**가 엄청 많아요. |\n\n### 대표 활용 분야:\n- **AI/머신러닝** (`TensorFlow`, `PyTorch`)\n- **데이터 분석** (`pandas`, `NumPy`)\n- **웹 개발** (`Django`, `Flask`)\n- **자동화 스크립트** (엑셀·PDF 처리, 웹 크롤링 등)\n- **교육** (코딩 입문 언어로 세계적으로 많이 채택됨)\n\n### 한 줄 예제:\n```python\nprint("Hello, world!")\n```\n→ 이 한 줄만으로도 화면에 글자가 찍혀요.\n\n요약하자면, 파이썬은 **“읽기 쉽고, 쓰기 쉽고, 강력한”** 언어라서 **개발자들이 가장 많이 쓰는 언어 top 3**에 꾸준히 들어가요.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 520, 'prompt_tokens': 35, 'total_tokens': 555, 'completion_tokens_details': None, 'prompt_token

#### API 키를 명시적으로 설정하는 방법

In [ ]:
from langchain_openai import ChatOpenAI

# 명시적으로 API 키 설정
#llm = ChatOpenAI(model="gpt-3.5-turbo", api_key=OPENAI_API_KEY)  
llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

response = llm.invoke([{"role": "user", "content": "파이썬이란?"}])
print(response)

In [5]:
# 그라디오 라이브러리를 불러옵니다.
import gradio as gr
# LangChain 의 ChatOpenAI 를 불러옵니다.
from langchain_openai import ChatOpenAI

# ChatOpenAI 인스턴스 생성
#llm = ChatOpenAI(model="gpt-3.5-turbo")
llm = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    #model="meta-llama/llama-4-scout-17b-16e-instruct",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)

# 채팅봇의 응답을 처리하는 함수를 정의합니다.
def chat_respond(message, chat_history):  
    response = llm.invoke([{"role": "user", "content": message}])
    bot_message = response.content #str
    
    chat_history.append({"role": "user", "content": message})
    chat_history.append({"role": "assistant", "content": bot_message})

    # 수정된 채팅 기록을 반환합니다.
    return "", chat_history  

# gr.Blocks()를 사용하여 인터페이스를 생성합니다.
with gr.Blocks() as demo:  
    chatbot = gr.Chatbot(label="채팅창")  # '채팅창'이라는 레이블을 가진 채팅봇 컴포넌트를 생성합니다.
    msg = gr.Textbox(label="입력")  # '입력'이라는 레이블을 가진 텍스트박스를 생성합니다.
    clear = gr.Button("초기화")  # '초기화'라는 레이블을 가진 버튼을 생성합니다.

    # 텍스트박스에 메시지를 입력하고 제출하면 respond 함수가 호출되도록 합니다.
    msg.submit(chat_respond, [msg, chatbot], [msg, chatbot])  
    # '초기화' 버튼을 클릭하면 채팅 기록을 초기화합니다.
    clear.click(lambda: None, None, chatbot, queue=False)  

# 인터페이스를 실행합니다. 실행하면 사용자는 '입력' 텍스트박스에 메시지를 작성하고 제출할 수 있으며, '초기화' 버튼을 통해 채팅 기록을 초기화 할 수 있습니다.
demo.launch(debug=True)  

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Keyboard interruption in main thread... closing server.


In [ ]:
demo.close()